<a href="https://colab.research.google.com/github/AlbertoCazzaniga/DTU_SUMMER_SCHOOL_DAY2/blob/main/DTU_SUMMER_SCHOOL_DAY2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DTU SUMMER SCHOOL: ADVANCED MACHINE LEARNING
## LAB SESSION ON INTRINSIC DIMENSION AND NEIGHBORHOOD OVERLAP ANALYSIS FOR REPRESENTATIONS FROM SELF-SUPERVISION

### Copenaghen, Tuesday 22/08/2023



Alberto Cazzaniga: alberto.cazzaniga@areasciencepark.it

Alessio Ansuini: alessio.ansuini@areasciencepark.it

---

## **PROBLEM 1: PROTEIN LANGUAGE MODELS REPRESENTATIONS**


Dropbox folder with data available: https://www.dropbox.com/scl/fo/tav4bk5smn1zxf4zyoetn/h?rlkey=yc0d7iubctfkc0p82ddukwr3g&dl=0

Given two ESM models, we will focus on understanding how the neighboorhod structure is changing between their corresponding embeddings by means of the Neighborhood Overlap method.

In particular, we want to understand if the neighboors composition is conserved or not in proximity of the *plateau* and the *peak* phase of the ID curve Figure 1.


<figure>
<center>
<img src='https://drive.google.com/uc?export=view&
id=1SctShFfLXNUDGPZJJQ8HKBa_6uyM7f5j' width="400" />
<figcaption> Figure 1: ID curve of ESM2 models. </figcaption></center>
</figure>

More formally:

- consider two models $X, Y \in$ {ESM2-35M, ESM2-650M, ESM2-3B};

- for a given dataset we compare the embeddings at layer $l$ of the model $X$ and at layer $m$ of the model $Y$;

- fix an integer $k$ that represent the "size" of the neighborhood we consider;

- compute neighborhood overlap between layer $l$ (of $X$) and $m$ (of $Y$) through the formula:

$$\chi_k^{l,m} = \frac{1}{N} \sum_i \frac{1}{k} \sum_j {A^l}_{ij} A^m_{ij}$$


We will perform this operation for all pairs of layers of $X$ and $Y$, thus obtaining $depth(X) \times depth(Y)$ values that we will plot as an heatmap.

## Requirements:

*  `matplotlib`
*   `seaborn`
*   `numpy`
*  [DADApy](https://dadapy.readthedocs.io/en/latest/)
*  [ESM](https://github.com/facebookresearch/esm)



In [ ]:
#!pip install seaborn
#!pip install numpy
#!pip install matplotlib
!pip install git+https://github.com/sissa-data-science/DADApy
!pip install fair-esm

In [ ]:
from dadapy import Data
from dadapy import *

import torch
import esm

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 10]

### Choose the models you want to compare

In [ ]:
models = {'35M':['ESM2-35M', 12],
          '650M':['ESM2-650M', 33],
          '3B':['ESM2-3B', 36]}

In [ ]:
x_model = '650M'
y_model = '35M'

reps_x = np.arange(models[x_model][1]+1)
reps_y = np.arange(models[y_model][1]+1)

### How to perform inference with ESM model
(from https://github.com/facebookresearch/esm)

In [ ]:
# Load model
model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
batch_converter = alphabet.get_batch_converter()
model.eval()

In [ ]:
# Obtain the data
data = [("protein1", "MKTVRQERLKSIVRILERSKEPVSGAQLAEELSVSRQVIVQDIAYLRSLGYNIVATPRGYVLAGG"),
        ("protein2", "KALTARQQEVFDLIRDHISQTGMPPTRAEIAQRLGFRSPNAAEEHLKALARKGVIEIVSGASRGIRLLQEE"),
        ("protein3", "KALTARQQEVFDLIRDISQTKGVIEIVSGASRGIRLLQEE")]

batch_labels, batch_strs, batch_tokens = batch_converter(data)
batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)

# Choose which representations you want to extract
reps = [r for r in range(34)]

# Extract per-residue representations
with torch.no_grad():
    results = model(batch_tokens, repr_layers = reps)

# Extract last layer representation only
token_representations = results["representations"][33]

# Generate per-sequence representations via averaging
# NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
sequence_representations = []
for i, tokens_len in enumerate(batch_lens):
    sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))

### Load the neighbor indexes and compute neighboorhood overlap

For 5000 proteins (nsamples) chosen in the ProteinNet dataset, we pre-computed the embeddings of the ESM models (for time saving).

You can download the representations from all the layer of the $x\_model$ and $y\_model$ running the following block of code.

Of course you are welcome to independently try to mimic this.

In [ ]:
if x_model == '650M' or y_model == '650M':
  !wget https://www.dropbox.com/scl/fi/feynf2trfzal4lvhrkcfm/ESM2-650M.tar.gz?rlkey=zfkcm7xebgfn1zymehll6tz52&dl=1
  !tar -zxvf ESM2-650M.tar.gz?rlkey=zfkcm7xebgfn1zymehll6tz52&dl=1

elif x_model == '3B' or y_model == '3B':
  !wget https://www.dropbox.com/scl/fi/m6it8rlblfpek15zzqle8/ESM2-3B.tar.gz?rlkey=g6f9zv5tdzil2xt6j4ni7zwg8&dl=1
  !tar -zxvf ESM2-3B.tar.gz?rlkey=g6f9zv5tdzil2xt6j4ni7zwg8&dl=1

if  x_model == '35M' or y_model == '35M':
  !wget https://www.dropbox.com/scl/fi/bz7dipe6ma38pc5oic92x/ESM2-35M.tar.gz?rlkey=663j56iaq4v9yr98r03spboce&dl=1
  !tar -zxvf ESM2-35M.tar.gz?rlkey=663j56iaq4v9yr98r03spboce&dl=1

Neighboor overlap is computed using the indexes of the k-nearest neighoboor (k-NN) of protein in our dataset (where max_k is 50).

For time constraints these have been already computed, and can be downloaded running the cell below.

As before, you are more then welcome to try your favourite Nearest Neighbor estimator to compute k-NN indices for the representations.

In [ ]:
if x_model == '650M' or y_model == '650M':
  !wget https://www.dropbox.com/scl/fi/z8rcq18lih79lo585g7iv/ESM2-650M-idx.tar.gz?rlkey=ydgt8urwon2g3mffue4g4wyaj&dl=1
  !tar -zxvf ESM2-650M-idx.tar.gz?rlkey=ydgt8urwon2g3mffue4g4wyaj&dl=1

elif x_model == '3B' or y_model == '3B':
  !wget https://www.dropbox.com/scl/fi/rgiy194u15y4dtzd56dby/ESM2-3B-idx.tar.gz?rlkey=l2aq5nupr666s6dsu0mi52rbo&dl=1
  !tar -zxvf ESM2-3B-idx.tar.gz?rlkey=l2aq5nupr666s6dsu0mi52rbo&dl=1

if  x_model == '35M' or y_model == '35M':
  !wget https://www.dropbox.com/scl/fi/1wpl2odsg0kyjvwg4y1b9/ESM2-35M-idx.tar.gz?rlkey=qanvf3sp2fozfvqo1zaxcx06o&dl=1
  !tar -zxvf ESM2-35M-idx.tar.gz?rlkey=qanvf3sp2fozfvqo1zaxcx06o&dl=1

Once you have downloades the indexes for each proteins you can write the NO function that takes as input 2 of these arrays.

### Write the function to calculate the neighboor overlap between two embeddings:

Since we provide only the first k = 50 nearest neighbors for each point, one has to set k less or equal than 50 for the function to work.

In [ ]:
def neig_overlap(X, Y, K = 10):
#     """Computes the neighborhood overlap between two representations.
#     Parameters
#     ----------
#     indices1 : 2D array of ints
#         nearest neighbor index matrix of the first representation
#     indices2 : 2D array of ints
#         nearest neighbor index matrix of the second representation
#     k : int
#         number of nearest neighbors used to compute the overlap

#     Returns
#     -------
#     overlap : float
#         neighborhood overlap between the two representations
#     """
#     assert indices1.shape[0] == indices2.shape[0]
#     ndata = indices1.shape[0]
#     overlaps = np.empty(ndata)

#     for i in range(ndata):
#         overlaps[i] = ...
#     return np.mean(overlaps)

### Loop over all the reps and obtain the final result:



You can run instead of your function the DADapy method that requires as input the 2 embeddings.

In [ ]:
# Create a dictionary where to store the results
overlaps = {k: [] for k in reps_x}

# Define the number of neighboor you want to consider
k = 10

In [ ]:
# Loop over all layer of model X
for x in reps_x:

        # Load x neig indexes
        idx_x = np.load(f'{models[x_model][0]}-idx/{models[x_model][0]}_rep{x}_5000-idx.npy')

        # Loop over all layers of model Y
        for y in reps_y:

            # Load y neig indexes
            idx_y = np.load(f'{models[y_model][0]}-idx/{models[y_model][0]}_rep{y}_5000.npy')

            # You call here your function
            ov = neig_overlap(X = idx_x, Y = idx_y, K = k)

            overlaps[x].append(ov)


In [ ]:
# Collect everything together
all_overlaps = []
for rep in reps_x:
    all_overlaps.append(overlaps[rep])
all_overlaps = np.stack(all_overlaps)

In [ ]:
ax = sns.heatmap(data = all_overlaps,
                annot = False,
                cmap = "viridis")

ax.set(xlabel = x_model,
       ylabel = y_model)

ax.set_title(r'$\chi^{l,m}$')

plt.show()

### Download the neighborhood overlaps already computed for ESM2-3B and ESM2-650M.

In [ ]:
# Download already computed overlaps and plot them
!wget https://www.dropbox.com/scl/fi/gno1iigb26u0nwkkjwy6z/overlap.tar.gz?rlkey=f1nun3m0yul72604wo6feilx4&dl=1
!tar -zxvf overlap.tar.gz?rlkey=f1nun3m0yul72604wo6feilx4&dl=1

Plot the results.

In [ ]:
file = './overlap/3B_650M_10_5000.npy'
xlab = file.split('/')[-1].split('_')[1]
ylab = file.split('/')[-1].split('_')[0]

ovs = np.load(file)
ax = sns.heatmap(data = ovs[1:,1:],
                annot = False,
                cmap = "viridis")

ax.set(xlabel = xlab,
       ylabel = ylab)

ax.set_title(r'$\chi^{l,m}$')

plt.show()

All we did so far was aimed to get to understand the basics in order to tackle this final analysis.

Using neighborhood overlap, discuss similarities and differences between representations of the ESM2 models (8M,32M,150M,650M,3B) focusing on:

- influence of the relative scale (i.e. relative number of parameters) of the models;
- influence of the position of layers within the network (e.g. layer number);
- relations with the intrinsic dimension graph in Figure 1.

---

# **iGPT lab session**



## **Comparision intrinsic dimension profiles between iGPT and iBERT**



<figure>
<center>
<img src='https://drive.google.com/uc?export=view&
id=16dwmsUi5LFSf_Vc7LHfFMZ3RdIpmfyKs' width="400" />
<figcaption> Figure 2: ID curve of iGPT-large. </figcaption></center>
</figure>



Figure 2 shows the intrinsic dimension (ID) profile of the ImageNet dataset in iGPT large. The scope of this section is to plot the ID on a subset of the iBERT representations.

### **Overview: the two nearest neighbor estimator (TwoNN)**

The TwoNN estimator measures the ID of a dataset with the ratios between the distances of each point to its second and first nearest neighbor.

$$\mu_i = \dfrac{r_{i_2}}{r_{i_1} }$$

The likelihood distribtuion of the $μ_i$ follows a Parteo law parametrized by the intrinsic dimension $d$ of the data.


$$p(\mu_i|d) = \dfrac{d}{\mu_i^{d+1}}$$

<!-- <figure>
<center>
<img src='https://drive.google.com/uc?export=view&
id=10x2A-oixnyc2gx_kihxh-uKOIM-EcEsP' width="400" />
<figcaption> Figure 2: ID curve of iGPT-large. </figcaption></center>
</figure> -->





Given the $μ_i$ from the $N$ datapoints the intrinsic dimension $d$ can be estimated by maximum likelihood with the formula:


$$    \hat{d} = \dfrac{N}{\sum_{i=1}^N \text{log} \mu_i} $$



### **Download data**
Download the distances from each data point to its first and second neighbor:

In [ ]:
%%capture
!gdown --id 1JFiKWYOl9TmPISJ872n1X3TarfWR-Tzl
!unzip -o igpt_data.zip
!rm igpt_data.zip

Inspect folder structure

In [ ]:
!apt-get install tree &> /dev/null
!tree -d igpt_data

### Load distance matrices ibert representations:

The distances from each data representations to its first two nearest neighbors are stored in **igpt_data/ibert/post_mlp**.

In this folder, you find the nearest neighbor distances of the the outputs of the multi-layer perceptron after layer normalization.

You find the distance matrix of the layers $\{1, 3, 5, 7, ... 49 \}$. You can load the distance matrix of, e.g., the layer "1" with:

```
folder = "igpt_data/ibert/post_mlp"
distances = np.load(folder +"/layer1_post_mlp_dist.npy")
```
The first column of the distance matrix, distances[:, 0], is full of zeros as it contains the distances form a datapoint to itself. distances[:, 1], distances[:, 2] contain the distances between each data points and its first and second neighbor respectively.
```
print(distances)

[[0, 0.12, 0.17]
[0, 0.12, 0.17]
...
[0, 0.16, 0.16]]
```





### **Compute the intrinsic dimension of the iBERT representations.**
Implement the TwoNN intrinsic dimension estimate of the iBERT representations with the formula provided above:

#### **Important note**

When computing the ID an important euristic is used. Facco et al. recommend removing the 10% of $\mu_i$ with the largest values.

You can do this as follows:

```
# N ->  number of datapoints
# mu_fraction = 0.9 -> number of mus to keep in the estimate
N_eff = int(N * mu_fraction)
mus_reduced = np.sort(mus)[:N_eff]
```
Then use **N_eff** and **mus_reduced** in the maximum likelihood esimate of the ID provided above.


**1. Implement the TwoNN estimator**
- Compute the $μ_i$ ratios from the distance matrix.
- Use the $μ_i$ to compute the ID.

**2. Compare how N_eff affects the ID estimate.**
- How does the ID profile change if we use all the $μ_i$?

**3. Plot the ID profiles of iBERT and compare to those of iGPT shown above.**







1. Implement the TwoNN estimator below


In [ ]:
# def compute_id_2nn(distances, mu_fraction = 0.9):+
#     """TwoNN intrinsic dimension estimator.

#     Parameters
#     ----------
#     distances : 2D array of floats
#         nearest neighbor distance matrix
#     mu_fraction : float
#         fraction of mus to consider in the ID estimate (see explanation above)

#     Returns
#     -------
#     intrinsic_dim : float
#         intrinsic dimension estimated with TwoNN
#     """
#     mus =                       #mu_ratios
#     N =                         #number of datapoints
#     N_eff =                     #fraction of datapoints used to compute the ID
#     mus_reduced =               #mus kept to estimated ID
#     intrisic_dim =              #id estimate
#     return intrinsic_dim

2. Compute the ID on the iBERT representations; try to see how the euristic on the $\mu$'s affect the ID estimate.

In [ ]:
#layers = np.arange(1, 50, 2)
#intrinsic_dimensions = []
#for layer in layers:
#  distances = np.load(f"igpt_data/ibert/post_mlp/layer{layer}_post_mlp_dist.npy")
#   intrinsic_dimensions.append(compute_id_2nn(distances, mu_fraction = ...))


3. Plot the ID of the bert representations and compare with those of iGPT shown above

In [ ]:
## write here the code of your plot

### **Optional: Exploring intrinsic dimension profiles in other layers of iGPT**

In *Valeriani et al.* the various statistics for iGPT are measured **after the first normalization layer** of each attention block. In this section, we want to explore how the profile changes if we measure the ID and the overlap in other positions of the attention blocks: after the normalization layer next to the self attention block, and before the first layer normalization.

The structure of the attention blocks in iGPT and iBERT is as follows:

1. Layer Noramlization
2. Self Attention
3. Layer Normalization
4. Multi-layer Perceptron


Compare the intrinsic dimenstion profiles in iGPT with the represerntation estracted:
1. **before** the first normalization layer;
2. **after** the first normalization layer (this is what you alredy computed above);
3. **after** the second normalization layer.

You can load the nearest neighbor distance matrices for these representations as:

```
#before the first normalization layer
distances = np.load(f"igpt_data/igpt/pre_ln/layer{lay}_post_mlp_dist.npy")

#after the first normalization layer
distances = np.load(f"igpt_data/igpt/post_mlp/layer{lay}_post_mlp_dist.npy")

#after the second normalization layer
distances = np.load(f"igpt_data/igpt/post_attn/layer{lay}_post_mlp_dist.npy")
```


## **Similarity of iBERT and iGPT representations**

In this section we will compare the representations of the ImageNet dataset at identical layers of the iGPT and iBERT networks. We will use $\chi_k^{l_{GPT}, l_{BERT} }$, the neighborhood overlap between pairs of representations. $\chi_k^{l_{GPT}, l_{BERT} }$ measures the fraction of common neighbors within the first $k$ averaged over the dataset.


Given the two adjacency lists of lenght $k$ in two represenatations $l_{BERT}$, $l_{GPT}$, the overlap *for a datapoint $i$* is computed as:


$$\chi_{k_i}^{l_{GPT}, l_{BERT}} = \dfrac{1}{k} \|A_{k_i}^{l_{BERT}} \cap A_{k_i}^{l_{GPT}} \|$$

where $\|A_i^{l_{BERT}} \cap A_i^{l_{GPT}} \|$ denotes the number of common elements between the two adjacency lists.

The neighborhood overlap between two representations is:
$$\chi_{k}^{l_{GPT}, l_{BERT}} = \dfrac{1}{N} \sum_i \chi_{k_i}^{l_{GPT}, l_{BERT}}$$

In the following we will measure the overlap between pairs layers **at the same depth** of iGPT and iBERT.

To do so, we use as input the indices of the first 15 neighbors of each data point.

You can load the nearest neighbor indices for,  e.g., layer 1 as follows:

```
indices_igpt = np.load("igpt/post_mlp/layer1_post_mlp_indices.npy")
indices_ibert = np.load("ibert/post_mlp/layer1_post_mlp_indices.npy")
```
the same holds for the other layers.

The structure of **indices_igpt** is as follows:

```
print(indices_igpt)
[[0, 8398 ...]
[1, 6289, ...]
[2, 12688, ..]
...
[19998, 1231, ... ]
[19999, 1266, ... ]]
```
thus, each datapoint is considered the first nearest neighbor of itself.

**Important note:**

In order to compute correctly the overlap between representations you must **remove the first element of each row**.


**Implement the function that computes overlap between representations here:**


In [ ]:
# def compute_data_overlaps(indices1, indices2, k=15):
#     """Computes the neighborhood overlap between two representations.

#     Parameters
#     ----------
#     indices1 : 2D array of ints
#         nearest neighbor index matrix of the first representation
#     indices2 : 2D array of ints
#         nearest neighbor index matrix of the second representation
#     k : int
#         number of nearest neighbors used to compute the overlap

#     Returns
#     -------
#     overlap : float
#         neighborhood overlap between the two representations
#     """

#     assert indices1.shape[0] == indices2.shape[0]
#     ndata = indices1.shape[0]
#     overlaps = np.empty(ndata)

#     for i in range(ndata):
#         overlaps[i] = ...
#     return np.mean(overlaps)

**Compute the data overlap between iBERT end iGPT for the layers provided below**:

In [ ]:
# layers = np.arange(1, 50, 2)
# overlap_igpt_ibert = []
# for lay in layers:
#     print(lay)
#     indices_igpt = np.load(f"igpt_data/igpt/post_mlp/layer{lay}_post_mlp_indices.npy")
#     indices_ibert = np.load(f"igpt_data/ibert/post_mlp/layer{lay}_post_mlp_indices.npy")
#     ...

**Plot the similarity between representations measured by the overlap**

## **Semantics of iBERT and iGPT representations**
The neighborhood overlap can be used as a kNN-like measure of how explicit a given representation is in terms of an external partition of the data. For the ImageNet dataset the external partition is the partition given by the class labels.

The class labels can be loaded with:

```
class_labels = np.load("igpt_data/labels.npy")
print(class_labels)
np.array([1, 1, 1, ..., 305, 305, 305])
```
You are give the same subset of the ImageNet training set you used to compute the similarity between iBERT and iGPT representations. This subset contains 20 thousands data point grouped in 100 classes with 200 images each.

The overlap computation follows the same guidelines given above with the following modifications:
1. the elements of the adjacency lists are the class labels of the neighbors.
2. you must count the number of neighbors with the same class labels of the central point and put this number in place of $\|A_i^{l_{BERT}} \cap A_i^{l_{GPT}} \|$


**Implement the function that computes overlap with the ground truth labels for the iBERT representations here**

In [ ]:
# def compute_label_overlap(indices, labels, k=15):
#     """Computes the neighborhood overlap with ground truth labels.

#     Parameters
#     ----------
#     indices1 : 2D array (int)
#         nearest neighbor index matrix of the first representation
#     labels : 1D array (int)
#         labels of the datapoints
#     k : int
#         number of nearest neighbors used to compute the overlap

#     Returns
#     -------
#     overlap : float
#         neighborhood overlap with ground truth labels
#     """
#     neighbor_index = indices[:, 1 : k + 1]
#     ...
#     overlaps = np.mean(overlaps)
#     return overlaps

**Compute the data overlap with the gorund truth labels for the iBERT representations here**

In [ ]:
# layers = np.arange(1, 50, 2)
# labels = np.load(f"igpt_data/labels.npy")
# k = 10
# ov_ibert = []
# for lay in layers:
#     print(lay)
#     indices_ibert = np.load(f"igpt_data/ibert/post_mlp/layer{lay}_post_mlp_indices.npy")
#     ov_ibert.append(compute_label_overlap(indices_ibert, labels, k))

**Plot the overlap with ground truth labels for iBERT**